# Neural Spike Sorting in Calcium Imaging Data I (Sparsity)

## Looking Ahead

We are going to download the CaImAn package for processing calcium imaging data, load some saved processed sample data provided with this package, and take a look at some aspects of it, including its sparsity and amenability to a convolutional signal model. We gave a motivating summary of this demo in the homework handout---be sure to read it carefully to understand things like where this dataset is coming from, how it is collected, and what kinds of preprocessing we skipped over.

The CaImAn package can be found [here](https://github.com/flatironinstitute/CaImAn). CaImAn is licensed under the GNU GPL, which you can find a copy of at the previous link; we make heavy use of code from the CaImAn `demos` subfolder in this notebook. You can read about the package and its capabilities in the associated paper [here](https://doi.org/10.1101/339564). We are using data processed with the CNMF-e algorithm, which you can read more about [here](https://elifesciences.org/articles/28728).



## Installation

We download a frozen version of the package that I prepared for this demo. We install a few extra python packages that we will need as well. We finally download a file with a processed version of the `data_endoscope.tif` file provided with the CaImAn package using the notebook `demo_pipeline_cnmfE.ipynb` also provided with the CaImAn package. You can check out that notebook at the CaImAn Github link above if you want to learn more about how the data is processed.

The installation may take a while (a minute or two). You should see `Successfully installed caiman` in the output window once it has finished.

In [ ]:
## Download CaImAn
#!rm -rf CaImAn
!git clone https://github.com/hwagyesa/CaImAn.git
  
## Install packages we need

## Hardcoded from environment.yml (based on what's missing)
!pip install ipyparallel
!pip install peakutils
!pip install pims
!pip install tifffile

## The data file is located at CaImAn/proc_data/cnmf_export.hdf5.

## Install Caiman
!pip install scikit-learn=='0.23.2'
!pip install h5py=='2.9.0'
!pip install Bokeh=='1.4.0'
import os
os.chdir('CaImAn')
!pip install -e .

## Loading the Processed Data

We load in the processed data and inspect it with some of the visualization tools provided by CaImAn.

In [ ]:
## Take care of imports
## We take these from the CaImAn demo notebook mentioned in the intro
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
#    get_ipython().magic(u'matplotlib qt')
except:
    pass
  
import numpy as np
import matplotlib.pyplot as plt
import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl

## Get data in memory.
## We are loading the processed image data, and the extracted neuron locations/traces.
cnm = cm.source_extraction.cnmf.cnmf.load_CNMF('/content/CaImAn/proc_data/cnmf_export.hdf5')
Yr, dims, T = cm.load_memmap('/content/CaImAn/proc_data/memmap__d1_128_d2_128_d3_1_order_C_frames_1000_.mmap')
images = Yr.T.reshape((T,) + dims, order='F')
cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=cnm.params.init['gSig'][0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
#cnm.estimates.evaluate_components(images, cnm.params)

## Visualizing the Data

Now that we have the data in memory, we can visualize it using built-in CaImAn functions. We take these from the CaImAn demo notebook.

### Videos of the Dataset

The videos below show how the endoscope data looks after postprocessing (sped up). In each video, the leftmost pane is the original (motion corrected) data, the middle pane is the reconstructed data, and the rightmost pane is the residual (difference between the first two panes).


#### With Background

The source extraction (demixing) step of the processing pipeline identifies a background component of the video, corresponding to non-noise and non-action-potential signal. This video shows what the data looks like *with* the background.

In [ ]:
import base64
from IPython.display import HTML

## Create the movie (save it to disk)
cnm.estimates.play_movie(images, q_max=99.5, magnification=2,
                         include_bck=True, gain_res=10,
                         bpx=0, the_backend='notebook')
plt.gcf().clear()

## Load the movie into memory and play it in a HTML element
video = open('/media/out.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
<video controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
</video>'''.format(encoded.decode('ascii')))



#### Without Background

Here is the same video of the data with the background component subtracted.

In [ ]:
import base64
from IPython.display import HTML

## Create the movie (save it to disk)
cnm.estimates.play_movie(images, q_max=99.9, magnification=1,
                         include_bck=False, gain_res=4, bpx=0,
                         the_backend='notebook')
plt.gcf().clear()

## Load the movie into memory and play it in a HTML element
video = open('/media/out.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
<video controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
</video>'''.format(encoded.decode('ascii')))

### Aggregate View of Accepted and Rejected Components

The source extraction step of the processing pipeline identifies candidate (spatial) regions in the video that correspond to locations where action potentials are present. After processing is finished, a final step of the pipeline applies threshold detection and a convolutional neural network classifier to determine which of these candidate regions really correspond to action potential signals, and which are spurious. The figure below visualizes the results of this classification process.

In [ ]:
#%% plot contour plots of accepted and rejected components
bpl.output_notebook()
plt.set_cmap('viridis')
cnm.estimates.plot_contours(img=cn_filter, idx=cnm.estimates.idx_components)

### Accepted Components (Spatial Locations and Temporal Traces)

Continuing from the previous figure, we take a more detailed view of the accepted spatial components. Below, the interactive plot allows us to view each accepted component's temporal trace: the action potential corresponding to the selected neuron. In the trace plot on the right, we see the deconvolved fluorescence signal for each neuron.

In [ ]:
bpl.output_notebook()
cnm.estimates.nb_view_components(img=cn_filter, idx=cnm.estimates.idx_components,
                                denoised_color='red', cmap='viridis')

## Understanding the Processed Data Contents

The results of CaImAn's data processing are contained in the `cnm.estimates` object that we were using above. The key signals (contained in this object) that we will need to use to inspect the data further are listed below.
- `idx_components`: Numpy array of indices corresponding to the accepted components. These are spatial components that were judged to correspond to neurons.
- `idx_components_bad`: Like the previous item, but these are the indices of the components that were judged to be spurious.
- `S`: A matrix whose rows are the deconvolved spiking signals corresponding to each of the components found. This signal is important for assessing the sparsity of the fluorescence signals. **Note**: This signal, and subsequent ones, have data for both accepted and rejected components. You must explicitly index using `idx_components` in order to look at only the non-spurious spiking signals.
- `C`:  A matrix whose rows are the denoised fluorescence signals corresponding to each of the components found. Each row of this matrix is the signal corresponding to the convolution of the AR1 model kernel (see below) and the spiking signal `S`. We plotted this signal in the previous figure: it corresponds to the red traces.
- `neuron_sn`: A vector with one element for each of the found components; its values correspond to the AR1 kernel parameters for each component's fluorescence signal. **Note**: Per the CaImAn documentation, it seems that these parameters are supposed to be stored in the variable `g`, but a bug returns them as `neuron_sn` instead (c.f. [this line](https://github.com/flatironinstitute/CaImAn/blob/468428fa9aab257cb6361bf8ad422467b58a0e68/caiman/source_extraction/cnmf/temporal.py#L64) and [here and the next few lines](https://github.com/flatironinstitute/CaImAn/blob/cf67385b2c0760043d69880e05729756bd698f2e/caiman/source_extraction/cnmf/cnmf.py#L705) for the relevant code based on the demo notebook we used to process the data).

## Your Tasks

Each level three header below contains a task you should complete. See the homework handout for additional details.

### Task 1: Understanding the Fluorescence Signal Model

In this task, we implement the AR1 kernel model and a convolution operation in order to regenerate the fluorescence signal `C` for one of the components we found above from the constituent parameters. This will help us to understand the convolution process, and thus give us the basic understanding necessary to implement a deconvolution algorithm of our own next time.

Below, let `c` denote the fluorescence trace for the first accepted component `C[idx_components[0], :]`.  Perform the following tasks:
1. Code up, or track down, an algorithm for performing convolution that matches the specification we gave in the homework handout: in particular, it needs to be a *bilinear operator* on the space of length-`len(c)` signals. You can implement the boundary conditions in any way you like (e.g., periodic extension, truncation, ...) as long as it satisfies the properties specified in the homework handout.
2. Write code to generate the AR1 kernel from the relevant parameter `neuron_sn[idx_components[0]]`. See the derivation in the homework handout.
3. To make sure your codes above is correct, plot the impulse response by generate the impulse signal of length `len(c)` (signal of the form (1, 0, 0, ...)), and convolve it with your AR1 kernel. What you observe should be the kernel itself. 
3. Use your convolution function together with the kernel from step 2 and the spike signal `S[idx_components[0], :]` to re-generate the fluorescence trace `c`.

The **outputs** for the above tasks should be: 
1. a plot of the impulse response of the system. 
2. a plot of the spike signal `S[idx_components[0], :]`
2. a plot of your regenerated fluorescence signal 
3. a print statement of an `l2` error estimate between the regenerated signal and the actual signal `c`.

In [ ]:
c=cnm.estimates.C[cnm.estimates.idx_components[0], :]
len(c)


In [ ]:
import scipy

In [ ]:
def conv_scipy(x,y):
  output = scipy.signal.convolve(x,y, mode='full', method='auto')
  return output

In [ ]:
AR1_coeff=cnm.estimates.neuron_sn[cnm.estimates.idx_components[0]]
AR1=[AR1_coeff[0]**i for i in range(0,len(c))]

In [ ]:
impulse =np.zeros(len(c))
impulse[0]=1
impulse_response=conv_scipy(impulse,AR1)

bpl.output_notebook()
h = bpl.figure(title="Impulse Response of the system")
h.xaxis.axis_label = 'time t'
h.yaxis.axis_label = 'AR1'
h.line(np.arange(1000),impulse_response)
bpl.show(h)

In [ ]:
bpl.output_notebook()
h = bpl.figure(title="Spike Signal")
h.xaxis.axis_label = 'time t'
h.yaxis.axis_label = 'Signal S(t)'
h.line(np.arange(1000),cnm.estimates.S[cnm.estimates.idx_components[0], :])
bpl.show(h)

In [ ]:
signal_s=cnm.estimates.S[cnm.estimates.idx_components[0], :]
reconstructed_signal = conv_scipy(signal_s,AR1)
reconstructed_signal=reconstructed_signal[:1000]
bpl.output_notebook()
h = bpl.figure(title="Reconstructed fluroscence Signal")
h.xaxis.axis_label = 'time t'
h.yaxis.axis_label = 'Signal C*(t)'
h.line(np.arange(1000),reconstructed_signal)
bpl.show(h)

In [ ]:
MSE=np.linalg.norm(np.subtract(c,reconstructed_signal))
print(f'The L2 error between the reconstructed and original signal is - {MSE}')

### Basic Questions About the Results

Provide answers to the following questions about the fluorescence signal `c` studied in the previous task:
1. Is the AR1 kernel you generated above sparse? How do the sparsities of `S[idx_components[0], :]` and of `c` compare?
2. From the plot above, describe perceptually how the reconstructed signal c is generated by the spike signal `S` and AR1 kernel. Do you think AR1 kernel captures the structure of the fluorescence signal? 

**Answer**
1. The AR1 kernel produced in the sense that the firsst few values of the kernel are of significant value and other terms are very miniscule that are near zero. The spike signal S is sparse is nature with roughly 20 non-zero values out of the total length 1000, whereas the signal c is well populated with most of the values to be non-zeros,well-distributed which makes the data not sparse in nature.
2.The reconstucted C is generated by convolving the signal S with kernel. The fluroscence is actually spike signal that is being seen and that would still persists after the spike for some time period t. This is perfectly captured by the exponentially decaying kernel hence this gives us a good estimate of the actual signal that is being received i.e. C. 